In [81]:

import xlwings as xw
import os
import pandas as pd
import numpy as np
pd.options.display.max_rows = None
pd.options.display.max_columns = None

In [82]:
path = 'C:/Users/nisha/OneDrive - Floating Solutions Consulting/Documents/04. Projects/JAD-01 MV Main Deck/03. Working/FMECA/Batch 5 FMECA 2022/FMECA spreadsheets/AN_working'
nameFile = 'KCH_chunk2.xlsx'
file_path = os.path.join(path,nameFile)
book = xw.Book(file_path)


In [83]:
sht = book.sheets['Sheet1']
rng = sht.range('A1:BO680')
df = rng.options(pd.DataFrame, omdex=False, header=True).value

In [84]:
df.head(1)

,Report reference,Anomaly ID,Anomaly Type,Frame(s),Stiffener(s),Structural Component,Length (X),Width/Height\n(Y),"Nearest transverse member (Fr., TWF,TBHD)",Longitudinal\n(l) (distance\nfrom in mm),"Nearest longitudinal member (SS,ObLBHD)",Transverse (t) (distance from in mm),Grade,Weight\n(kgs),As Built Thickness\n(mm),Max Allowable Diminution (%),Renewal Thickness (mm),Substantial Corer Thickness (mm),EN01,EN02,EN03,EN04,EN05,EN06,EN07,EN08,EN09,EN10,Average UTM Reading (mm),Minimum \nUTT \nReading,(mm),(%),FLAG,Primary Structure,Secondary Structure,Local Structure,Detail Structure,Anomaly Nature,Concatenate,Sev,Occ,Det,RPN\npre-action,Adj Occ pre,Adj\nRPN \npre-action,Required Engineering Action,Has engineering action been completed ?,Generic action (following engineering action),Sev2,Occ2,Det2,RPN\npost-action,Adj Occ post,Adj\nRPN \npost-action,Avg % diminution,Anom extents,Stress level (location on span),Proximity / density of anom.,Stress / buckling calc?,Final Anomaly description,Location,Detail,Repair,Inspection interval,Assessment notes,Adj Final action
Item No.,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1.0,VER-52531-4SWBT-LBHD-CR-0001-1379,4SWBT-LBHD- CR-701,AW,FR-48,LL20,LBHD(S),100.0,70.0,FR-49,170.0,LL20,340.0,AH,0.9,15.5,20.0,12.4,13.2,12.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.2,9.35,6.2,6.15,0.396774,R,Long'l bhd,Panel,Plate,None,Corrosion,Long'l bhdPanelPlateCorrosion,5.0,7.0,8.0,280.0,None,0.0,Investigate further and perform calculation if...,No,Arrest & monitor CVI,5.0,3.0,4.0,60.0,NaN,0.0,0.396774,Local,Med,Med,Y,Generalised corrosion,Tank boundary plating,Between 6 - 10mm remaining,Permanent repair,Scheduled tank inspection,Local pitting on longitudinal bulkhead plating...,Permanent repair


In [85]:
df.shape

(679, 66)

In [86]:
abt_lbhd = {"LL00":14,
"LL01":14,
"LL02":14,
"LL03":14,
"LL04":11.5,
"LL05":11.5,
"LL06":11.5,
"LL07":11.5,
"LL08":12,
"LL09":12,
"LL10":12,
"LL11":12,
"LL12":13,
"LL13":13,
"LL14":13,
"LL15":13,
"LL16":14.5,
"LL17":14.5,
"LL18":14.5,
"LL19":14.5,
"LL20":15,
"LL21":15,
"LL22":15,
"LL23":16,
"LL24":16,
"LL25":16,
"LL26":19.5,
"LL27":19.5,
}

In [87]:
def abt_check(row):
    if row['Local Structure'] == 'Plate':
        if row['Stiffener(s)'] in abt_lbhd:
            if abt_lbhd[row['Stiffener(s)']] == row["As Built Thickness\n(mm)"]:
                return 'abt correct'
            else:
                return 'check abt'
        

In [88]:
def correct_abt(row):
    if row['Local Structure'] == 'Plate':
        if row['Stiffener(s)'] in abt_lbhd:
            if abt_lbhd[row['Stiffener(s)']] == row["As Built Thickness\n(mm)"]:
                return 'abt correct'
            else:
                return abt_lbhd[row['Stiffener(s)']]
        

In [89]:
df['abtCheck'] = df.apply(abt_check, axis=1)
df['correctAbt'] = df.apply(correct_abt, axis=1)

In [90]:
correct_abts = df[df['abtCheck']=='check abt'][['Anomaly ID','Stiffener(s)','As Built Thickness\n(mm)','abtCheck','correctAbt']]

xw.view(correct_abts)

In [91]:
df.head(3)

,Report reference,Anomaly ID,Anomaly Type,Frame(s),Stiffener(s),Structural Component,Length (X),Width/Height\n(Y),"Nearest transverse member (Fr., TWF,TBHD)",Longitudinal\n(l) (distance\nfrom in mm),"Nearest longitudinal member (SS,ObLBHD)",Transverse (t) (distance from in mm),Grade,Weight\n(kgs),As Built Thickness\n(mm),Max Allowable Diminution (%),Renewal Thickness (mm),Substantial Corer Thickness (mm),EN01,EN02,EN03,EN04,EN05,EN06,EN07,EN08,EN09,EN10,Average UTM Reading (mm),Minimum \nUTT \nReading,(mm),(%),FLAG,Primary Structure,Secondary Structure,Local Structure,Detail Structure,Anomaly Nature,Concatenate,Sev,Occ,Det,RPN\npre-action,Adj Occ pre,Adj\nRPN \npre-action,Required Engineering Action,Has engineering action been completed ?,Generic action (following engineering action),Sev2,Occ2,Det2,RPN\npost-action,Adj Occ post,Adj\nRPN \npost-action,Avg % diminution,Anom extents,Stress level (location on span),Proximity / density of anom.,Stress / buckling calc?,Final Anomaly description,Location,Detail,Repair,Inspection interval,Assessment notes,Adj Final action,abtCheck,correctAbt
Item No.,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1.0,VER-52531-4SWBT-LBHD-CR-0001-1379,4SWBT-LBHD- CR-701,AW,FR-48,LL20,LBHD(S),100.0,70.0,FR-49,170.0,LL20,340.0,AH,0.9,15.5,20.0,12.4,13.2,12.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.2,9.35,6.2,6.15,0.396774,R,Long'l bhd,Panel,Plate,None,Corrosion,Long'l bhdPanelPlateCorrosion,5.0,7.0,8.0,280.0,None,0.0,Investigate further and perform calculation if...,No,Arrest & monitor CVI,5.0,3.0,4.0,60.0,NaN,0.0,0.396774,Local,Med,Med,Y,Generalised corrosion,Tank boundary plating,Between 6 - 10mm remaining,Permanent repair,Scheduled tank inspection,Local pitting on longitudinal bulkhead plating...,Permanent repair,check abt,15
2.0,VER-52531-4SWBT-LBHD-CR-0001-1379,4SWBT-LBHD- CR-702,AW,FR-48,LL20,LBHD(S),120.0,350.0,FR-49,0.0,LL20,60.0,AH,5.1,15.5,20.0,12.4,13.2,11.8,12.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.10,11.8,3.40,0.219355,R,Long'l bhd,Panel,Plate,None,Corrosion,Long'l bhdPanelPlateCorrosion,5.0,7.0,8.0,280.0,None,0.0,Investigate further and perform calculation if...,No,Arrest & monitor CVI,5.0,3.0,4.0,60.0,5.0,100.0,0.219355,Local,Med,Med,Y,Generalised corrosion,Tank boundary plating,More than 10mm remaining and greater than rene...,investigate,None,Local corrosion on longitudinal bulkhead stif...,Arrest & monitor CVI,check abt,15
3.0,VER-52531-4SWBT-LBHD-CR-0001-1379,4SWBT-LBHD- CR-703,AW,FR-48,LL20,LBHD(S),200.0,100.0,FR-48,730.0,LL20,0.0,AH,1.6,10.5,25.0,7.9,8.5,6.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.20,6.2,4.30,0.409524,R,Long'l bhd,Panel,Plate,None,Pitting,Long'l bhdPanelPlatePitting,8.0,7.0,8.0,448.0,None,0.0,Investigate further and perform calculation if...,No,Arrest & monitor CVI,8.0,3.0,4.0,96.0,NaN,0.0,0.409524,Local,Med,Med,N,Pit,Tank boundary plating,None,None,None,Local corrosion on longitudinal bulkhead plati...,Monitor only,check abt,15


In [92]:
def finalAnomalyDescription(row):
    if row['Final Anomaly description'] == 'Pit within generalised corrosion':
        return 'Pit within generalised corrosion'
    else:
        if row['Anomaly Nature'] == 'Corrosion':
            return 'Generalised corrosion'
        elif row['Anomaly Nature'] == 'Pitting':
            return 'Pit'
        else:
            return ''

In [93]:
df['final_anomlay_description'] = df.apply(finalAnomalyDescription,axis=1)

In [94]:
def locationStructure(row):
    if row['Local Structure'] == 'Plate':
        return 'Tank boundary plating'
    elif row['Local Structure'] == "Long'l stiff'r":
        if row['Detail Structure'] == 'Web':
            return 'Stiffener web'
        elif row['Detail Structure'] == 'Flange':
            return 'Stiffener flange middle 1/3'
    else:
        return ''

In [95]:
df['location_formulated'] = df.apply(locationStructure,axis=1)

In [96]:
df.head(1)

,Report reference,Anomaly ID,Anomaly Type,Frame(s),Stiffener(s),Structural Component,Length (X),Width/Height\n(Y),"Nearest transverse member (Fr., TWF,TBHD)",Longitudinal\n(l) (distance\nfrom in mm),"Nearest longitudinal member (SS,ObLBHD)",Transverse (t) (distance from in mm),Grade,Weight\n(kgs),As Built Thickness\n(mm),Max Allowable Diminution (%),Renewal Thickness (mm),Substantial Corer Thickness (mm),EN01,EN02,EN03,EN04,EN05,EN06,EN07,EN08,EN09,EN10,Average UTM Reading (mm),Minimum \nUTT \nReading,(mm),(%),FLAG,Primary Structure,Secondary Structure,Local Structure,Detail Structure,Anomaly Nature,Concatenate,Sev,Occ,Det,RPN\npre-action,Adj Occ pre,Adj\nRPN \npre-action,Required Engineering Action,Has engineering action been completed ?,Generic action (following engineering action),Sev2,Occ2,Det2,RPN\npost-action,Adj Occ post,Adj\nRPN \npost-action,Avg % diminution,Anom extents,Stress level (location on span),Proximity / density of anom.,Stress / buckling calc?,Final Anomaly description,Location,Detail,Repair,Inspection interval,Assessment notes,Adj Final action,abtCheck,correctAbt,final_anomlay_description,location_formulated
Item No.,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1.0,VER-52531-4SWBT-LBHD-CR-0001-1379,4SWBT-LBHD- CR-701,AW,FR-48,LL20,LBHD(S),100.0,70.0,FR-49,170.0,LL20,340.0,AH,0.9,15.5,20.0,12.4,13.2,12.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.2,9.35,6.2,6.15,0.396774,R,Long'l bhd,Panel,Plate,None,Corrosion,Long'l bhdPanelPlateCorrosion,5.0,7.0,8.0,280.0,None,0.0,Investigate further and perform calculation if...,No,Arrest & monitor CVI,5.0,3.0,4.0,60.0,NaN,0.0,0.396774,Local,Med,Med,Y,Generalised corrosion,Tank boundary plating,Between 6 - 10mm remaining,Permanent repair,Scheduled tank inspection,Local pitting on longitudinal bulkhead plating...,Permanent repair,check abt,15,Generalised corrosion,Tank boundary plating


In [97]:
def detailStatus(row):
    remaining_t = row['Average UTM Reading (mm)']
    loss = row['(mm)']
    renewable_loss = row["As Built Thickness\n(mm)"] - row["Renewal Thickness (mm)"]
    substantial_loss = row["As Built Thickness\n(mm)"] - row["Substantial Corer Thickness (mm)"]
    if row['Final Anomaly description'] == 'Pit':
        if row['Location'] == 'Tank boundary plating':
            if remaining_t > 10.0 and loss > renewable_loss:
                return 'More than 10mm remaining and greater than renewable loss'
            elif 6.0 < remaining_t and  remaining_t <= 10.0:
                return 'Between 6 - 10mm remaining'
            elif remaining_t < 6.0:
                return 'Less  than 6mm remaining'
            else:
                return 'investigate'
        if row['Location'] == 'Stiffener web':
            if loss > renewable_loss:
                return 'Any loss greater than renewable'
            else:
                return 'investigate'
        if row['Location'] == 'Stiffener flange middle 1/3':
            if loss > renewable_loss:
                return 'Any loss greater than renewable'
            else:
                return 'investigate'
        if row['Location'] == 'Stiffener flange outside of middle 1/3':
            if remaining_t > 6.0 and loss > renewable_loss:
                return 'More than 6mm remaining and greater than renewable loss'
            elif remaining_t <= 6.0:
                return 'Less than 6mm remaining'
    if row['Final Anomaly description'] == 'Generalised corrosion':
        if remaining_t > 10.0 and loss > renewable_loss:
            return 'More than 10mm remaining and greater than renewable loss'
        elif 6.0 < remaining_t and  remaining_t <= 10.0:
            return 'Between 6 - 10mm remaining'
        elif remaining_t < 6.0:
            return 'Less  than 6mm remaining'
        else:
            return 'investigate'
    if row['Final Anomaly description'] == 'Pit within generalised corrosion':
        if remaining_t > 10.0 and loss > renewable_loss:
            return 'More than 10mm remaining and greater than renewable loss'
        elif 6.0 < remaining_t and  remaining_t <= 10.0:
            return 'Between 6 - 10mm remaining'
        elif remaining_t < 6.0:
            return 'Less  than 6mm remaining'
        else:
            return 'investigate'



    

In [98]:
def repairMethod(row):
    detail = row['Detail']
    if row['Final Anomaly description'] == 'Pit':
        if row['Location'] == 'Tank boundary plating':
            if detail == 'More than 10mm remaining and greater than renewable loss':
                return 'investigate'
            elif detail == 'Between 6 - 10mm remaining':
                return 'Arrest and monitor CVI'
            elif detail == 'Less  than 6mm remaining':
                return 'Permanent repair'
            else:
                return 'investigate'
        if row['Location'] == 'Stiffener web':
            if detail == 'Any loss greater than renewable':
                return 'investigate coating condition'
            else:
                return 'investigate coating condition'
        if row['Location'] == 'Stiffener flange middle 1/3':
            if detail == 'Any loss greater than renewable':
                return 'Arrest and monitor CVI'
            else:
                return 'investigate'
        if row['Location'] == 'Stiffener flange outside of middle 1/3':
            if detail == 'More than 6mm remaining and greater than renewable loss':
                return 'A&M CVI or investigate'
            elif detail == 'Less than 6mm remaining':
                return 'investigate'
    if row['Final Anomaly description'] == 'Generalised corrosion':
        if detail == 'More than 10mm remaining and greater than renewable loss':
            return 'investigate'
        elif detail == 'Between 6 - 10mm remaining':
            return 'Arrest and monitor CVI'
        elif detail == 'Less  than 6mm remaining':
            return 'Permanent repair'
        else:
            return 'investigate'
    if row['Final Anomaly description'] == 'Pit within generalised corrosion':
        if detail == 'More than 10mm remaining and greater than renewable loss':
            return 'investigate'
        elif detail == 'Between 6 - 10mm remaining':
            return 'Arrest and monitor CVI'
        elif detail == 'Less  than 6mm remaining':
            return 'Permanent repair'
        else:
            return 'investigate'

In [99]:
df['detail_formulated'] = df.apply(detailStatus,axis=1)

In [100]:
df['repair_formulate'] = df.apply(repairMethod,axis=1)

In [101]:
df.head(1)

,Report reference,Anomaly ID,Anomaly Type,Frame(s),Stiffener(s),Structural Component,Length (X),Width/Height\n(Y),"Nearest transverse member (Fr., TWF,TBHD)",Longitudinal\n(l) (distance\nfrom in mm),"Nearest longitudinal member (SS,ObLBHD)",Transverse (t) (distance from in mm),Grade,Weight\n(kgs),As Built Thickness\n(mm),Max Allowable Diminution (%),Renewal Thickness (mm),Substantial Corer Thickness (mm),EN01,EN02,EN03,EN04,EN05,EN06,EN07,EN08,EN09,EN10,Average UTM Reading (mm),Minimum \nUTT \nReading,(mm),(%),FLAG,Primary Structure,Secondary Structure,Local Structure,Detail Structure,Anomaly Nature,Concatenate,Sev,Occ,Det,RPN\npre-action,Adj Occ pre,Adj\nRPN \npre-action,Required Engineering Action,Has engineering action been completed ?,Generic action (following engineering action),Sev2,Occ2,Det2,RPN\npost-action,Adj Occ post,Adj\nRPN \npost-action,Avg % diminution,Anom extents,Stress level (location on span),Proximity / density of anom.,Stress / buckling calc?,Final Anomaly description,Location,Detail,Repair,Inspection interval,Assessment notes,Adj Final action,abtCheck,correctAbt,final_anomlay_description,location_formulated,detail_formulated,repair_formulate
Item No.,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1.0,VER-52531-4SWBT-LBHD-CR-0001-1379,4SWBT-LBHD- CR-701,AW,FR-48,LL20,LBHD(S),100.0,70.0,FR-49,170.0,LL20,340.0,AH,0.9,15.5,20.0,12.4,13.2,12.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.2,9.35,6.2,6.15,0.396774,R,Long'l bhd,Panel,Plate,None,Corrosion,Long'l bhdPanelPlateCorrosion,5.0,7.0,8.0,280.0,None,0.0,Investigate further and perform calculation if...,No,Arrest & monitor CVI,5.0,3.0,4.0,60.0,NaN,0.0,0.396774,Local,Med,Med,Y,Generalised corrosion,Tank boundary plating,Between 6 - 10mm remaining,Permanent repair,Scheduled tank inspection,Local pitting on longitudinal bulkhead plating...,Permanent repair,check abt,15,Generalised corrosion,Tank boundary plating,Between 6 - 10mm remaining,Arrest and monitor CVI


In [102]:
summary_df_cols = ['Report reference', 'Anomaly ID','Stiffener(s)','Nearest longitudinal member (SS,ObLBHD)',
       'As Built Thickness\n(mm)', 
       'Renewal Thickness (mm)', 'Substantial Corer Thickness (mm)',
       'Average UTM Reading (mm)', 'Minimum \nUTT \nReading ', '(mm)', '(%)',
       'FLAG','correctAbt',
       'final_anomlay_description',
       'location_formulated', 'detail_formulated', 'repair_formulate']

In [78]:
xw.view(df[summary_df_cols])

In [79]:
xw.view(df)

In [103]:
xw.view(df.groupby(['repair_formulate']).count())

In [ ]:
df.groupby(['repair_formulate']).count().reset_index()

,repair_formulate,Report reference,Anomaly ID,Anomaly Type,Frame(s),Stiffener(s),Structural Component,Length (X),Width/Height\n(Y),"Nearest transverse member (Fr., TWF,TBHD)",Longitudinal\n(l) (distance\nfrom in mm),"Nearest longitudinal member (SS,ObLBHD)",Transverse (t) (distance from in mm),Grade,Weight\n(kgs),As Built Thickness\n(mm),Max Allowable Diminution (%),Renewal Thickness (mm),Substantial Corer Thickness (mm),EN01,EN02,EN03,EN04,EN05,EN06,EN07,EN08,EN09,EN10,Average UTM Reading (mm),Minimum \nUTT \nReading,(mm),(%),FLAG,Primary Structure,Secondary Structure,Local Structure,Detail Structure,Anomaly Nature,Concatenate,Sev,Occ,Det,RPN\npre-action,Adj Occ pre,Adj\nRPN \npre-action,Required Engineering Action,Has engineering action been completed ?,Generic action (following engineering action),Sev2,Occ2,Det2,RPN\npost-action,Adj Occ post,Adj\nRPN \npost-action,Avg % diminution,Anom extents,Stress level (location on span),Proximity / density of anom.,Stress / buckling calc?,Final Anomaly description,Location,Detail,Repair,Inspection interval,Assessment notes,Adj Final action,batch,abtCheck,correctAbt,final_anomlay_description,location_formulated,detail_formulated
0,Arrest and monitor CVI,79,79,79,79,79,79,79,79,78,75,79,75,79,69,79,79,79,79,79,57,33,18,5,4,1,1,1,1,79,79,79,79,79,79,79,79,2,79,79,79,79,79,79,0,43,79,76,79,79,79,79,79,0,43,79,79,79,79,79,79,79,79,79,79,79,79,79,45,45,79,79,79
1,Permanent repair,7,7,7,7,7,7,7,7,7,6,7,6,7,6,7,7,7,7,7,7,4,3,3,3,3,3,2,1,7,7,7,7,7,7,7,7,0,7,7,7,7,7,7,0,5,7,7,7,7,7,7,7,0,5,7,7,7,7,7,7,7,7,7,7,7,7,7,5,5,7,7,7
2,investigate,911,912,912,912,912,912,908,912,908,874,911,836,911,888,912,912,912,912,880,588,473,357,270,141,103,84,59,39,880,911,880,880,781,912,912,912,79,912,912,912,912,912,912,0,594,912,296,911,912,912,912,912,17,598,881,432,432,432,432,912,800,324,324,324,432,431,912,465,465,912,903,912
3,investigate coating condition,122,122,122,122,122,122,122,122,122,121,122,26,122,106,122,122,122,122,122,78,42,23,12,4,1,1,1,1,122,122,122,122,121,122,122,122,122,122,122,122,122,122,122,0,14,122,32,122,122,122,122,122,0,14,122,118,118,118,118,122,122,116,116,116,118,118,122,0,0,122,122,122
